In [1]:
import torch.nn as nn
import torch.nn.functional as F
import net_size_utils as nsu


class Net(nn.Module):
    def __init__(self, kernel_size=5):
        super().__init__()
        self.kernel_size = kernel_size
        self.conv1 = nn.Conv2d(3, 6, kernel_size)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size)
        conv_height, conv_width = self.conv_out_size(224, 224)
        self.fc1 = nn.Linear(16 * conv_width * conv_height, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def conv_out_size(self, height, width):
        dim = (height, width)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        return dim


In [2]:
from birds_dataset import Birds270Dataset

import net_training
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

dataset_dir = "../data/birds270"
selected_birds = ["ALBATROSS", "BALD EAGLE", "BARN OWL", "EURASIAN MAGPIE", "FLAMINGO",
                  "MALLARD DUCK", "OSTRICH", "PEACOCK", "PELICAN", "TRUMPTER SWAN"]
train_transform= transforms.Compose([
    transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5))
])
test_transform = transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5)) # normalizes colors to range [-1,1]
train_dataset = Birds270Dataset(dataset_dir, set_type="train", selected_birds=selected_birds, transform=train_transform)
test_dataset = Birds270Dataset(dataset_dir, set_type=["test","valid"], selected_birds=selected_birds, transform=test_transform)

label_set = test_dataset.get_label_set()

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

net = Net()
net_training.train_and_evaluate(net, train_dataloader, test_dataloader, label_set, epochs=5, print_results=True)


Epoch 0:
	train loss: 1.8752480690890103
	validation loss: 1.3121366453170777, validation accuracy: 55.00000000000001%
	Elapsed time: 0:00:10.234544
Epoch 1:
	train loss: 1.1060602636856955
	validation loss: 0.8524828243255616, validation accuracy: 74.0%
	Elapsed time: 0:00:10.621304
Epoch 2:
	train loss: 0.6727419245401082
	validation loss: 0.7064674067497253, validation accuracy: 76.0%
	Elapsed time: 0:00:10.285876
Epoch 3:
	train loss: 0.38386595249176025
	validation loss: 0.5668010818958282, validation accuracy: 81.0%
	Elapsed time: 0:00:11.257515
Epoch 4:
	train loss: 0.173803426687426
	validation loss: 0.5798923349380494, validation accuracy: 80.0%
	Elapsed time: 0:00:10.673650
Final loss: 0.5799, final accuracy: 80.00 %
	ALBATROSS: 60.00
	BALD EAGLE: 70.00
	BARN OWL: 80.00
	EURASIAN MAGPIE: 90.00
	FLAMINGO: 100.00
	MALLARD DUCK: 90.00
	OSTRICH: 90.00
	PEACOCK: 100.00
	PELICAN: 50.00
	TRUMPTER SWAN: 70.00


{'correct': 80,
 'total': 100,
 'accuracy': 0.8,
 'loss': 0.5798923349380494,
 'correct_labels': {'ALBATROSS': 6,
  'BALD EAGLE': 7,
  'BARN OWL': 8,
  'EURASIAN MAGPIE': 9,
  'FLAMINGO': 10,
  'MALLARD DUCK': 9,
  'OSTRICH': 9,
  'PEACOCK': 10,
  'PELICAN': 5,
  'TRUMPTER SWAN': 7},
 'total_labels': {'ALBATROSS': 10,
  'BALD EAGLE': 10,
  'BARN OWL': 10,
  'EURASIAN MAGPIE': 10,
  'FLAMINGO': 10,
  'MALLARD DUCK': 10,
  'OSTRICH': 10,
  'PEACOCK': 10,
  'PELICAN': 10,
  'TRUMPTER SWAN': 10},
 'label_accuracy': {'ALBATROSS': 0.6,
  'BALD EAGLE': 0.7,
  'BARN OWL': 0.8,
  'EURASIAN MAGPIE': 0.9,
  'FLAMINGO': 1.0,
  'MALLARD DUCK': 0.9,
  'OSTRICH': 0.9,
  'PEACOCK': 1.0,
  'PELICAN': 0.5,
  'TRUMPTER SWAN': 0.7}}